# Database Project (SWE3033) (Fall 2023)
# Homework #10 (50pts, Due date: 12/06)

**Instruction:** In this homework, we provide you with a jupyter notebook file (DBP_Homework10.ipynb). You should follow the instructions in these documents carefully.

**Submit two files as follows**:
- `DBP_Homework10_StudentID.zip`
	- `DBP_Homework10_StudentID.ipynb`
	- `DBP_Homework10_StudentID.pdf`
   

In [115]:
!pip install langchain
!pip install langchainhub
!pip install google-generativeai
!pip install tiktoken
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.18
    Uninstalling urllib3-1.26.18:
      Successfully uninstalled urllib3-1.26.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kubernetes 28.1.0 requires urllib3<2.0,>=1.24.2, but you have urllib3 2.1.0 which is incompatible.
  Using cached urllib3-1.26.18-py2.py3-none-any.whl (143 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.1.0
    Uninstalling urllib3-2.1.0:
      Successfully uninstalled urllib3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
types-requests 2.31.0.10 requires urllib3>=2, but you have urllib3 1.26.18 which is incomp

#### Before submitting, you must delete the GOOGLE_API_KEY

In [117]:
import bs4
from operator import itemgetter

from langchain import hub
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import GooglePalmEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.chat_models import ChatGooglePalm
from langchain.schema.runnable.history import RunnableWithMessageHistory
from langchain.prompts import ChatPromptTemplate

# 1.

In [118]:
# Load the document
loader = WebBaseLoader(web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",), bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),)
raw_documents = loader.load()

# ============== EDIT HERE ==================
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=800, # Maximum size of chunks to return
  chunk_overlap=100 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(raw_documents)

# Embed each chunks and load it into vector store
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GooglePalmEmbeddings(google_api_key=GOOGLE_API_KEY)
)

# Retrieve
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={"k": 6}
)

# Generate
llm = ChatGooglePalm(google_api_key=GOOGLE_API_KEY)
# ===========================================

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

query = "What is Task Decomposition?"

In [119]:
# a.	Langchian hub - rlm/rag-prompt
# ============== EDIT HERE ==================
prompt = hub.pull("rlm/rag-prompt")
# ===========================================
print("## Prompt template ##")
print("Input variables: ", prompt.input_variables)
print("Template: ", prompt.messages[0].prompt.template)
print()

# ============== EDIT HERE ==================
rag_chain = (
  {"context": retriever | format_docs,"question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)
# ===========================================

print("## Generated string ##")
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

## Prompt template ##
Input variables:  ['context', 'question']
Template:  You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:

## Generated string ##
Task decomposition is the process of breaking down a complex task into smaller, more manageable tasks. This can be done manually or automatically. Manual task decomposition involves breaking down the task into smaller steps and then assigning each step to a different person or team. Automatic task decomposition involves using a computer program to break down the task into smaller steps.

There are several benefits to task decomposition. First, it can help to improve efficiency by making it easier to manage and track progress. Second, it can help to improve quality by ensuring that each step is completed co

In [120]:
# b.	Langchain hub – gregkamradt/test-question-making
# ============== EDIT HERE ==================
prompt = hub.pull("gregkamradt/test-question-making")
# ===========================================
print("## Prompt template ##")
print("Input variables: ", prompt.input_variables)
print("Template: ", prompt.template)
print()

# ============== EDIT HERE ==================
rag_chain = (
  {"context": retriever | format_docs,"question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)
# ===========================================

print("## Generated string ##")
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

## Prompt template ##
Input variables:  ['context', 'question']
Template:  Your goal is to create a well crafted set of answers for a test for a specific question.
Your answers will be used on a test to access a students knowledge. You will be given the question and your goal is to follow the output format below w/ guidelines

Answer Choice Guidelines
Position the correct option so that it appears approximately the same number of times in each possible position for a quiz. 
Answer choices should be written clearly and similarly to each other in content, length, and grammar; avoid giving clues through the use of faulty grammatical construction. 
Make all distractors plausible; they should be common misconceptions that learners may have.
In answer choices, avoid “all of the above” and “none of the above," which can lead to artificially higher levels of performance.
In answer choices, avoid references to answer choices by letter (ie: “Both A and B”), as our answers are randomized
When num

In [121]:
# c.	Create new prompt to help your QA service
# ============== EDIT HERE ==================
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_template("""
  You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
  If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
  Question: {question}
  Context: {context}
  Answer:
"""
)

# ===========================================
#print("## Prompt template ##")
#print("Input variables: ", prompt.input_variables)
#print("Template: ", prompt.template)
#print()

# ============== EDIT HERE ==================
rag_chain = (
  {"context": retriever | format_docs,"question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)
# ===========================================

print("## Generated string ##")
for chunk in rag_chain.stream(query):
    print(chunk, end="", flush=True)

## Generated string ##
Task decomposition is the process of breaking down a complex task into smaller, more manageable tasks. This can be done by identifying the subtasks that need to be completed, and then ordering them in a logical way. Task decomposition can be helpful for planning and executing complex tasks, and it can also be used to improve communication and collaboration.

In the context of LLMs, task decomposition can be used to improve the performance of LLMs on complex tasks. By breaking down a complex task into smaller, more manageable tasks, LLMs can be trained to perform each task more effectively. This can lead to improved performance on the overall task.

There are a number of different ways to decompose a task. One common approach is to use a top-down approach. This involves starting with the overall task, and then breaking it down into smaller and smaller subtasks until each subtask is relatively simple. Another common approach is to use a bottom-up approach. This inv

# 2.

In [122]:
del llm, prompt, rag_chain

In [123]:
vectorstore.delete_collection()

In [124]:
# Load the document
loader = WebBaseLoader(web_paths=("https://asana.com/resources/project-stakeholder",))
raw_documents = loader.load()

# Same with question 1
# ============== EDIT HERE ==================
# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=800, # Maximum size of chunks to return
  chunk_overlap=100 # Overlap in characters between chunks
)
splits = text_splitter.split_documents(raw_documents)

# Embed each chunks and load it into vector store
vectorstore = Chroma.from_documents(
  documents=splits,
  embedding=GooglePalmEmbeddings(google_api_key=GOOGLE_API_KEY)
)

# Retrieve
retriever = vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={"k": 6}
)

# Generate
llm = ChatGooglePalm(google_api_key=GOOGLE_API_KEY)
# ===========================================

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [125]:
planner_prompt_template = "I want to do the project management for {input}"
planner_prompt = ChatPromptTemplate.from_template(planner_prompt_template)

# ============== EDIT HERE ==================
# Planner
# Use `from_template` method
planner = (
    planner_prompt
    | llm
    | StrOutputParser()
)
# ===========================================

In [126]:
internal_prompt_template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following: {base}
"""
internal_prompt = ChatPromptTemplate.from_template(internal_prompt_template)

# ============== EDIT HERE ==================
# Internal
# Use `from_template` method
# Use `question`, `retriever`, `format_docs` for `context`
internal = (
    {"context": itemgetter("question1") | retriever | format_docs, "question": itemgetter("question1"), "base": planner}
    | internal_prompt
    | llm
    | StrOutputParser()
)
# ===========================================

In [127]:
external_prompt_template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following: {base}
"""
external_prompt = ChatPromptTemplate.from_template(internal_prompt_template)

# ============== EDIT HERE ==================
# External, it is similar to `internal`
# Use `from_template` method
# Use `question`, `retriever`, `format_docs` for `context`
external =  (
    {"context": itemgetter("question2") | retriever | format_docs, "question": itemgetter("question2"), "base": planner}
    | internal_prompt
    | llm
    | StrOutputParser()
)
# ===========================================

In [128]:
# ============== EDIT HERE ==================
# Analysis
# Use `from_messages` method
analysis = (
    ChatPromptTemplate.from_messages(
        [
          ("system", "Generate a stakeholder analysis map for a given project."),
          ("human", "The details of a given project is following: {result3}"),
          ("ai", "Internal stakeholders:\n{result1}\n\nExternal stakeholders:\n{result2}")
        ]
    )
    | llm
    | StrOutputParser()
)
# ===========================================

In [129]:
# ============== EDIT HERE ==================
# Final responder
# Use `from_messages` method
final_responder = (
    ChatPromptTemplate.from_messages(
        [
          ("system", "Generate a final response given the information."),
          ("ai", "{a_response}"),
          ("human", "Common pitfalls:\n{a_pitfalls}")
        ]
    )
    | llm
    | StrOutputParser()
)
# ===========================================

In [130]:
# You can check that you implement `planner` correctly using this cell
planner.invoke({"input": "scrum"})

'Scrum is a framework for developing and managing complex products. It is a lightweight framework with a focus on iterative development, team collaboration, and accountability. Scrum teams are cross-functional and self-organizing, and they work in sprints of two weeks or less.\n\nThe Scrum Master is responsible for ensuring that the Scrum process is followed and that the team is able to deliver the product backlog. The Product Owner is responsible for prioritizing the product backlog and ensuring that the team is working on the most important features. The Development Team is responsible for developing the product according to the Scrum process.\n\nTo do project management for Scrum, you need to understand the Scrum framework and the roles and responsibilities of each team member. You also need to be able to manage the Scrum process, including sprint planning, sprint execution, and sprint retrospectives.\n\nHere are some tips for project management for Scrum:\n\n* **Understand the Scru

In [131]:
# ============== EDIT HERE ==================
# Chain 1
chain1 = internal
# ===========================================

# You can check that you implement `chain1`, `internal` correctly using this cell
chain1.invoke({"input": "scrum", "question1": "List the internal stakeholders."})

"Internal stakeholders are people within an organization who are affected by or have an interest in the outcome of a project. They can include:\n\n* Project manager\n* Project team members\n* Project sponsor\n* Executive leaders\n* Other cross-functional internal teams\n\nIt is important to identify and manage internal stakeholders effectively in order to ensure the success of a project. This can be done by:\n\n* Communicating regularly with stakeholders\n* Keeping stakeholders informed of progress and changes\n* Involving stakeholders in decision-making\n* Managing stakeholder expectations\n\nBy taking the time to manage internal stakeholders effectively, project managers can help to ensure that projects are completed on time, within budget, and to the satisfaction of all stakeholders.\n\nHere are some additional tips for managing internal stakeholders:\n\n* **Identify your stakeholders early on.** The first step in managing internal stakeholders is to identify who they are. This incl

In [132]:
# ============== EDIT HERE ==================
# Chain 2
chain2 = external
# ===========================================

# You can check that you implement `chain2`, `external` correctly using this cell
chain2.invoke({"input": "scrum", "question2": "List the external stakeholders."})

"External stakeholders are people or groups outside of the organization that are affected by the project or have an interest in its success. They can include customers, suppliers, partners, regulators, and the public.\n\nExternal stakeholders can play a significant role in the success of a project. They can provide feedback on the project, help to identify risks and opportunities, and advocate for the project within their own organizations. It is important to identify and manage external stakeholders early in the project lifecycle. This will help to ensure that their needs are met and that they are supportive of the project.\n\nThere are a number of ways to manage external stakeholders. One way is to create a stakeholder register. This is a document that lists all of the project's external stakeholders and their contact information. The stakeholder register should also include information on each stakeholder's interest in the project, their level of influence, and any potential risks o

In [133]:
# ============== EDIT HERE ==================
# Chain 3
# For `result3`, use the following line:
# "result3": itemgetter("question3") | retriever | format_docs
chain3 = (
    {
        "result3": itemgetter("question3") | retriever | format_docs,
        "result2": chain2,
        "result1": chain1
    }
    | analysis
)
# ===========================================

# You can check that you implement `chain3`, `analysis` correctly using this cell
chain3.invoke({
    "input": "scrum",
    "question1": "List the internal stakeholders.",
    "question2": "List the external stakeholders.",
    "question3": "How to do a stockholder analysis?"
    })

'Stakeholder analysis is a process of identifying and assessing the needs, interests, and potential impact of all the people or groups who may be affected by a project. It is an essential part of project management, as it helps to ensure that all stakeholders are considered and that their needs are met.\n\nStakeholders can be internal or external to the organization. Internal stakeholders are those who are directly involved in the project, such as the project manager, project team members, and project sponsor. External stakeholders are those who are not directly involved in the project, but who may be affected by its outcome, such as customers, suppliers, partners, and the public.\n\nThere are a number of benefits to conducting a stakeholder analysis. First, it helps to identify all of the stakeholders who may be affected by the project. This is important, as it ensures that all stakeholders are considered and that their needs are met. Second, it helps to assess the needs and interests

In [134]:
# Chain 4 (Don't modify)
chain4 = (
    {"question": itemgetter("question4"),
     "context": itemgetter("question4") | retriever | format_docs}
    | ChatPromptTemplate.from_template("{context} {question}")
    | llm
    | StrOutputParser()
)

chain4.invoke({"question4": "List the common stakeholder mapping pitfalls."})

"Here are the common stakeholder mapping pitfalls:\n\n* **Lack of boundaries:** Overeager project stakeholders can cause scope creep. To avoid this, implement a change control process.\n* **Not including stakeholders:** During the initial stakeholder identification phase, you forgot about a stakeholder who has a high interest in this initiative. To avoid this, involve your project team during the stakeholder identification process.\n* **Bringing stakeholders in too late:** You already kicked off your project, and then started your stakeholder analysis. To avoid this, create a stakeholder analysis map up front.\n* **Not communicating effectively:** You're not communicating effectively with your stakeholders, which is leading to misunderstandings and frustration. To avoid this, create a communication plan and stick to it.\n* **Not managing expectations:** You're not managing your stakeholders' expectations, which is leading to disappointment and frustration. To avoid this, set clear expe

In [135]:
# ============== EDIT HERE ==================
# Chain 5
chain5 = (
    {
        "a_response": chain3,
        "a_pitfalls": chain4
    }
    | final_responder
)
# ===========================================

# You can check that you implement `chain5`, `final_responder` correctly using this cell
chain5.invoke({
    "input": "scrum",
    "question1": "List the internal stakeholders.",
    "question2": "List the external stakeholders.",
    "question3": "How to do a stockholder analysis map?",
    "question4": "List the common stakeholder mapping pitfalls."
    })

"Stakeholder analysis is a critical part of project management. By understanding the needs and expectations of stakeholders, project managers can develop strategies to manage them effectively. However, there are a number of common pitfalls that can derail stakeholder management efforts.\n\nOne common pitfall is **not including stakeholders**. This can happen if stakeholders are not identified early in the project or if they are not given an opportunity to provide input. When stakeholders are not included, they may feel like they are not being heard and may become resistant to the project.\n\nAnother common pitfall is **not communicating effectively**. Stakeholders need to know what is going on with the project and they need to know it in a timely manner. If communication is not effective, stakeholders may become frustrated and may start to spread rumors or negative information about the project.\n\n**Not managing expectations** is another common pitfall. Stakeholders need to know what 

# Make sure to save both the code and output values prior to submission.
# You must delete your GOOGLE_API_KEY before submitting.